In [26]:
from math import sqrt
import pandas as pd
import numpy as np

df = pd.read_csv('telemetry-rio-5-laps.csv', decimal='.').loc[:, ['current_lap_time', 'position_x', 'position_y', 'position_z', 'lap_number', 'gear', 'speed', 'current_engine_rpm', 'fuel', 'wheel_on_rumble_strip_front_left', 'wheel_on_rumble_strip_front_right', 'wheel_on_rumble_strip_rear_left', 'wheel_on_rumble_strip_rear_right']]

first_x, first_y = 809.8262, 141.70494
second_x, second_y = -694.0124, 299.16605
thirdy_x, thirdy_y = 342.6821, 289.19034

radius = 125
first = True
second = False
thirdy = False
nextlap = 1

df['sector'] = None

for index, row in df.iterrows():
    if not pd.isnull(row['sector']):
        continue
    
    if first:
        if pow(row["position_x"] - second_x, 2) + pow(row["position_z"] - second_y, 2) < radius:
            first = False
            second = True
        
        df.at[index, "sector"] = "sector1"
        continue
            
    if second:
        if pow(row["position_x"] - thirdy_x, 2) + pow(row["position_z"] - thirdy_y, 2) < radius:
            second = False
            thirdy = True
        
        df.at[index, "sector"] = "sector2"
        continue
    
    if thirdy:
        if row['lap_number'] == nextlap:
            thirdy = False
            first = True
            nextlap += 1
            df.at[index, "sector"] = "sector1"
        else:
            df.at[index, "sector"] = "sector3"
        continue

df['current_section_time'] = df.groupby('sector')['current_lap_time'].transform(lambda x: x - x.iloc[0])

df.to_csv('CircuitData.csv')

In [14]:
import pandas as pd

# Carregar o arquivo CSV com as colunas necessárias
df = pd.read_csv('CircuitData.csv', decimal='.').loc[:, [
    'current_lap_time', 
    'position_x', 
    'position_y', 
    'position_z', 
    'lap_number', 
    'gear', 
    'speed', 
    'current_engine_rpm', 
    'fuel', 
    'wheel_on_rumble_strip_front_left', 
    'wheel_on_rumble_strip_front_right', 
    'wheel_on_rumble_strip_rear_left', 
    'wheel_on_rumble_strip_rear_right', 
    'sector'
]]

# Criar a nova coluna 'current_section_time' com base na separação por setor
df['current_section_time'] = df.groupby('sector')['current_lap_time'].transform(lambda x: x - x.iloc[0])

# Salvar o DataFrame atualizado de volta no arquivo CSV
df.to_csv('CircuitData.csv', index=False)


In [ ]:
df

In [2]:
import pandas as pd

df = pd.read_csv('CircuitData.csv')

velocidade_media_por_setor = df.groupby('sector')['speed'].mean().reset_index()

velocidade_media_por_setor.columns = ['Sector', 'Average_Speed']

velocidade_media_por_setor.to_csv('VelocidadeMediaPorSetor.csv', index=False)

print(velocidade_media_por_setor)


    Sector  Average_Speed
0  sector1      43.068725
1  sector2      36.404644
2  sector3      37.706250


In [15]:
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv('CircuitData.csv', decimal='.')

# Calcular o tempo de cada setor por volta
sector_times = df.groupby(['lap_number', 'sector'])['current_section_time'].max().reset_index()

# Renomear a coluna para indicar que se trata do tempo do setor
sector_times.rename(columns={'current_section_time': 'sector_time'}, inplace=True)

# Exibir o resultado
print(sector_times)

# Opcional: Salvar o resultado em um novo CSV
sector_times.to_csv('SectorTimesByLap.csv', index=False)


    lap_number   sector  sector_time
0            0  sector1   160.789260
1            0  sector2    76.168423
2            0  sector3    36.117535
3            1  sector1   158.154450
4            1  sector2    74.083649
5            1  sector3    33.457715
6            2  sector1   155.697560
7            2  sector2    72.735113
8            2  sector3    31.025815
9            3  sector1   158.829850
10           3  sector2    71.867173
11           3  sector3    34.124635
12           4  sector1   153.113240
13           4  sector2    69.884143
14           4  sector3    28.424835
